<div style="text-align: center;">
<h1>The University of North Carolina at Chapel Hill</h1>
<h1>Comp 555 BioAlgorithms - Spring 2018</h1>
<h1 style="font-size: 250%;">Problem Set #3 </h1>
<h1>Issued Thursday, 3/1/2018; Due Thursday, 3/22/2018</h1>
</div>

**Homework Information:** Some of the problems are probably too long to be started the night before the due date, so plan accordingly. Late problem sets will be penalized by a factor of	70.71% for each class meeting after the due date. Feel free to get help from others, but the work you submit in should be your own.

In [1]:
# Replace the following string values with the requested information
class Student:
    first = "Mohamed"
    last = "Lansari"
    onyen = "mlans13"
    pid = "730013359"

---
**Problem #1:** A file of *15-mers* simulating short reads from a genome can be downloaded [here](http://csbio.unc.edu/mcmillan/Comp555S18/data/kmers.txt). How many distinct nodes appear in the De Bruijn graph that represents these *15-mers* as edges? How many nodes are semi-balenced? How many nodes are balanced? How many are balanced with both in-degrees and out-degrees equal to 1?  

In [2]:
import itertools

# Build a basic graph description
class Graph:
    def __init__(self, vlist=[]):
        """ Initialize a Graph with an optional vertex list """
        self.index = {v:i for i,v in enumerate(vlist)}
        self.vertex = {i:v for i,v in enumerate(vlist)}
        self.edge = []
        self.edgelabel = []
    def addVertex(self, label):
        """ Add a labeled vertex to the graph """
        index = len(self.index)
        self.index[label] = index
        self.vertex[index] = label
    def addEdge(self, vsrc, vdst, label='', repeats=True):
        """ Add a directed edge to the graph, with an optional label. 
        Repeated edges are distinct, unless repeats is set to False. """
        e = (self.index[vsrc], self.index[vdst])
        if (repeats) or (e not in self.edge):
            self.edge.append(e)
            self.edgelabel.append(label)
    def hamiltonianPath(self):
        """ A Brute-force method for finding a Hamiltonian Path. 
        Basically, all possible N! paths are enumerated and checked
        for edges. Since edges can be reused there are no distictions
        made for *which* version of a repeated edge. """
        for path in itertools.permutations(sorted(self.index.values())):
            for i in xrange(len(path)-1):
                if ((path[i],path[i+1]) not in self.edge):
                    break
            else:
                return [self.vertex[i] for i in path]
        return []
    def SearchTree(self, path, verticesLeft):
        """ A recursive Branch-and-Bound Hamiltonian Path search. 
        Paths are extended one node at a time using only available
        edges from the graph. """
        if (len(verticesLeft) == 0):
            self.PathV2result = [self.vertex[i] for i in path]
            return True
        for v in verticesLeft:
            if (len(path) == 0) or ((path[-1],v) in self.edge):
                if self.SearchTree(path+[v], [r for r in verticesLeft if r != v]):
                    return True
        return False
    def hamiltonianPathV2(self):
        """ A wrapper function for invoking the Branch-and-Bound 
        Hamiltonian Path search. """
        self.PathV2result = []
        self.SearchTree([],sorted(self.index.values()))                
        return self.PathV2result
    def degrees(self):
        """ Returns two dictionaries with the inDegree and outDegree
        of each node from the graph. """
        inDegree = {}
        outDegree = {}
        for src, dst in self.edge:
            outDegree[src] = outDegree.get(src, 0) + 1
            inDegree[dst] = inDegree.get(dst, 0) + 1
        return inDegree, outDegree
    def verifyAndGetStart(self):
        inDegree, outDegree = self.degrees()
        start = 0
        end = 0
        for vert in self.vertex.iterkeys():
            ins = inDegree.get(vert,0)
            outs = outDegree.get(vert,0)
            if (ins == outs):
                continue
            elif (ins - outs == 1):
                end = vert
            elif (outs - ins == 1):
                start = vert
            else:
                start, end = -1, -1
                break
        if (start >= 0) and (end >= 0):
            return start
        else:
            return -1
    def eulerianPath(self):
        graph = [(src,dst) for src,dst in self.edge]
        currentVertex = self.verifyAndGetStart()
        path = [currentVertex]
        # "next" is where vertices get inserted into our tour
        # it starts at the end (i.e. it is the same as appending),
        # but later "side-trips" will insert in the middle
        next = 1
        while len(graph) > 0:
            for edge in graph:
                if (edge[0] == currentVertex):
                    currentVertex = edge[1]
                    graph.remove(edge)
                    path.insert(next, currentVertex)
                    next += 1
                    break
            else:
                for edge in graph:
                    try:
                        next = path.index(edge[0]) + 1
                        currentVertex = edge[0]
                        break
                    except ValueError:
                        continue
                else:
                    print "There is no path!"
                    return False
        return path
    def eulerEdges(self, path):
        edgeId = {}
        for i in xrange(len(self.edge)):
            edgeId[self.edge[i]] = edgeId.get(self.edge[i], []) + [i]
        edgeList = []
        for i in xrange(len(path)-1):
            edgeList.append(self.edgelabel[edgeId[path[i],path[i+1]].pop()])            
        return edgeList
    def render(self, highlightPath=[]):
        """ Outputs a version of the graph that can be rendered
        using graphviz tools (http://www.graphviz.org/)."""
        edgeId = {}
        for i in xrange(len(self.edge)):
            edgeId[self.edge[i]] = edgeId.get(self.edge[i], []) + [i]
        edgeSet = set()
        for i in xrange(len(highlightPath)-1):
            src = self.index[highlightPath[i]]
            dst = self.index[highlightPath[i+1]]
            edgeSet.add(edgeId[src,dst].pop())
        result = ''
        result += 'digraph {\n'
        result += '   graph [nodesep=2, size="10,10"];\n'
        for index, label in self.vertex.iteritems():
            result += '    N%d [shape="box", style="rounded", label="%s"];\n' % (index, label)
        for i, e in enumerate(self.edge):
            src, dst = e
            result += '    N%d -> N%d' % (src, dst)
            label = self.edgelabel[i]
            if (len(label) > 0):
                if (i in edgeSet):
                    result += ' [label="%s", penwidth=3.0]' % (label)
                else:
                    result += ' [label="%s"]' % (label)
            elif (i in edgeSet):
                result += ' [penwidth=3.0]'                
            result += ';\n'                
        result += '    overlap=false;\n'
        result += '}\n'
        return result
    
# function to read a file of k-mer edges, and use them to describe a graph
def graphFromEdgeFile(filename, k):
    grph = Graph()
    vert = set()
    edges = []
    # read in the file
    with open(filename, 'r') as f:
        for line in f:
            vert.add(line[0:k - 1])
            vert.add(line[1:k])
            edges.append(line[0:k])
    
    for v in vert:
        grph.addVertex(v)
    for e in edges:
        grph.addEdge(e[0:k - 1], e[1:k], e)
            
    return grph, edges, vert

# lets assess the degrees
g, edges, verts = graphFromEdgeFile("kmers.txt", 15)
inDeg, outDeg = g.degrees()

# information about our graph
print "We have", len(inDeg), "in degrees and", len(outDeg), "out-degrees"
print "We have", len(verts), "vertices and", len(edges), "edges"

# perform in- versus out-degree comparison
nBal = 0
nSemi = 0
inOutUno= 0

# generate the set of both in-degree and out-degree keys
keysTot = set()
for k in inDeg.keys():
    keysTot.add(k)
for k in outDeg.keys():
    keysTot.add(k)
    
# calculate the various styles of node
for e in keysTot:
    if inDeg.get(e, 0) == outDeg.get(e, 0):
        nBal += 1
        if inDeg.get(e, 0) == 1:
            inOutUno += 1
    elif abs(inDeg.get(e, 0) - outDeg.get(e, 0)) == 1:
        nSemi += 1

# print our results
print "The number of balanced nodes is", nBal
print "The number of semi-balanced nodes is", nSemi
print "The number of balanced nodes with in- and out-degree equal to one is", inOutUno

# calculate the eulerian path only once
ePath = g.eulerianPath()
print "The length of the graph's eulerian path is", len(ePath)

We have 93673 in degrees and 93673 out-degrees
We have 93674 vertices and 93925 edges
The number of balanced nodes is 93672
The number of semi-balanced nodes is 2
The number of balanced nodes with in- and out-degree equal to one is 93522
The length of the graph's eulerian path is 93926


---
**Problem #2:** What is the length of the Eulerian path that can be constructed in the De Bruijn graph described in Problem #1? How does the resulting constructed sequence compare to the plasmid sequence of [*Salmonella Typhimurium*](http://csbio.unc.edu/mcmillan/Comp555S18/data/SalmonellaTyphimurium.fa). (Hint: one method of comparison is consider how many k-mers the two sequences share. Of course you can compare where k=15, but consider what is the smallest value of k for which the two sequences differ in their set of k-mers?)

In [3]:
# opening fasta
# we need to begin by loading the genome
def loadFasta(filename):
    """ Parses a classically formatted and possibly 
        compressed FASTA file into a list of headers 
        and fragment sequences for each sequence contained"""
    if (filename.endswith(".gz")):
        fp = gzip.open(filename, 'rb')
    else:
        fp = open(filename, 'r')
    # split at headers
    data = fp.read().split('>')
    fp.close()
    # ignore whatever appears before the 1st header
    data.pop(0)     
    headers = []
    sequences = []
    for sequence in data:
        lines = sequence.split('\n')
        headers.append(lines.pop(0))
        # add an extra "+" to make string "1-referenced"
        sequences.append('+' + ''.join(lines))
    return (headers, sequences)

# start with the generated eulerian path
print len(ePath)

# build the actual path
eTrans = g.vertex[ePath[0]]
for n in xrange(1, len(ePath)):
    eTrans += g.vertex[ePath[n]][13]

print "The length of eTrans is", len(eTrans)
print "A slice of eTrans is", eTrans[200:300]

# now we need to calculate the smallest value of k for which the two sequences differ in their set of k-mers
headers, sequences = loadFasta("SalmonellaTyphimurium.fa")
salDNA = sequences[1][1:]   # this is the genome in question

print "salDNA from 0 to 100 is", salDNA[0:100]

# compile all the k-mers of each sequence, both the path and from the salmonella
# start at 15?
gMers = {}   # kmers for graph
sMers = {}   # kmers for salmonella
for k in xrange(3, len(eTrans)):
    # current k val
    print "the current k val is", k
    
    # generate the kmer sets
    gMers[k] = set()
    sMers[k] = set()
    
    # generate the kmer lists for each
    for i in xrange(len(eTrans) - k + 1):
        gMers[k].add(str(eTrans[i:i + k]))
    for i in xrange(len(salDNA) - k + 1):
        sMers[k].add(str(salDNA[i:i + k]))
        
    # check for the delta between the sets
    print "The length of gMers is", len(gMers[k]), "and the length of sMers is", len(sMers[k])
    print "The delta between them is", sMers[k] - gMers[k]
                
    if len(sMers[k] - gMers[k]) != 0:
        break
    

93926
The length of eTrans is 93939
A slice of eTrans is GTATCACAACGGTATTTTTGCCACCAGTCATTATGAAGGCGCGCTCACCAGTGAAGATATCAGTAGTGCGGCGGCCGCCGCCCTGAAAGGGCATAAAATC
salDNA from 0 to 100 is GAGTGAACGGATGAAACAGAAAGACCGTCTGTACGGCGTGGCACCGGCCTTACCCCGATTGCAGGCTGTGAAGCTAGGCCGCAGGTCCGCTATGTGTTTA
the current k val is 3
The length of gMers is 64 and the length of sMers is 64
The delta between them is set([])
the current k val is 4
The length of gMers is 256 and the length of sMers is 256
The delta between them is set([])
the current k val is 5
The length of gMers is 1024 and the length of sMers is 1024
The delta between them is set([])
the current k val is 6
The length of gMers is 4071 and the length of sMers is 4071
The delta between them is set([])
the current k val is 7
The length of gMers is 15230 and the length of sMers is 15230
The delta between them is set([])
the current k val is 8
The length of gMers is 41540 and the length of sMers is 41540
The delta between them is set([])
the current k val

---
**Problem #3:** You will find a BWT of the primary "genome" sequence of *Salmonella Typhimurium* [here](http://csbio.unc.edu/mcmillan/Comp555S18/data/SalmonellaTyphimurium.bwt). This BWT is compressed as follows: Any run of two or more repeated characters is prefixed with an ASCII-encoded number followed by the character. For example the string "AAACGGTTTTTTTTTT" would be encoded as the string "3AC2G10T". What is the compression ratio of this BWT? Where the compression ration is given by: 

$$\frac{len(compressed BWT)}{len(sequence)}$$

What is the average run-length in the BWT (consider characters without runs as run-lengths of 1)?

In [4]:
# USEFUL
codons = ["A", "T", "C", "G", "$"]

# we begin by loading in the BWT
bwt = ""
with open("SalmonellaTyphimurium.bwt", 'r') as f:
    bwt = f.read().replace("\n", '')

# function to parse the bwt's full length
def getExpansion(seq):
    ind = 0
    expLen = 0
    readLens = []
    while ind < len(seq):
        num = ""
        while seq[ind] not in codons:
            num = num + seq[ind]
            ind += 1
        if num != "":
            expLen += int(num)
            readLens.append(int(num))
        else:
            expLen += 1
            readLens.append(1)
        ind += 1
    
    return expLen, readLens

# get the expansion of the bwt
expansion, reads = getExpansion(bwt)

# calculate the average read length
avg = 0.0
for n in reads:
    avg += n
avg /= len(reads)

print "The original length of the bwt is", len(bwt)
print "The length of the expanded bwt is", expansion
print "This means that the compression ratio is", (len(bwt) / float(expansion))
print "The average read length of the bwt is", avg

The original length of the bwt is 4397999
The length of the expanded bwt is 4857433
This means that the compression ratio is 0.905416297044
The average read length of the bwt is 1.40828468415


**Problem #4:** Uncompress the BWT from in Problem #3, and use code given in class to find how many times the substring "ATGACAACGC" and its reverse complement "GCGTTGTCAT" appear in the *Salmonella Typhimurium* genome. Repeat this for the substring "ATGACAACGCAT" and its reverse complement "ATGCGTTGTCAT". Note these sequences are the first few bases of the *HolC* gene that you searched for in Problem Set #2.

In [45]:
# create a bwt transcription function
def getBWTTranscription(seq):
    ind = 0
    transcription = ""
    
    # iterate through the entire bwt
    while ind < len(seq):
        num = ""
        while seq[ind] not in codons:
            num += seq[ind]
            ind += 1
        if num != "":
            transcription += (seq[ind] * int(num))
        else:
            transcription += seq[ind]
        ind += 1
    return transcription

# method to generate an FMIndex
def FMIndex(bwt):
    fm = [{c: 0 for c in bwt}]
    for c in bwt:
        row = {symbol: count + 1 if (symbol == c) else count for symbol, count in fm[-1].iteritems()}
        fm.append(row)
    offset = {}
    N = 0
    for symbol in sorted(row.keys()):
        offset[symbol] = N
        N += row[symbol]
    return fm, offset

# method used to search bwt for substrings
def findBWT(pattern, FMIndex, Offset):
    lo = 0
    hi = len(FMIndex) - 1
    for symbol in reversed(pattern):
        lo = Offset[symbol] + FMIndex[lo][symbol]
        hi = Offset[symbol] + FMIndex[hi][symbol]
    return lo, hi

# get the transcription
trans = getBWTTranscription(bwt)

# generate an FMIndex for the BWT
fm, offset = FMIndex(trans)

# search for all occurences of "ATGACAACGC" and its reverse complement
lo1, hi1 = findBWT("ATGACAACGC", fm, offset)
print "We have", hi1 - lo1, "occurences of ATGACAACGC"
lo2, hi2 = findBWT("GCGTTGTCAT", fm, offset)
print "We have", hi2 - lo2, "occurences of GCGTTGTCAT, its reverse complement"

# search for all occurences of "ATGACAACGCAT" and its reverse complement
lo3, hi3 = findBWT("ATGACAACGCAT", fm, offset)
print "We have", hi3 - lo3, "occurences of ATGACAACGCAT"
lo4, hi4 = findBWT("ATGACAACGCAT"[::-1], fm, offset)
print "We have", hi4 - lo4, "occurences of ATGCGTTGTCAT, its reverse complement"

We have 4 occurences of ATGACAACGC
We have 13 occurences of GCGTTGTCAT, its reverse complement
We have 1 occurences of ATGACAACGCAT
We have 1 occurences of ATGCGTTGTCAT, its reverse complement


**Problem #5: Programming Problem** 

In class we discussed an algortihm to produce a BWT from a suffix array. In this problem you are asked to write code to do the inverse-- ***produce a suffix array from a BWT***. In the space provided below write your function and test it by finding the genomic indices for all of the substrings than you reported for Problem #4. (Hint: The first implict suffix in the BWT begins with '$', which is the last character of the original string).

In [50]:
# Enter your code here
codons = ["A", "T", "C", "G", "$"]

# we begin by loading in the BWT
def loadBWT():
    bwt = ""
    with open("SalmonellaTyphimurium.bwt", 'r') as f:
        bwt = f.read().replace("\n", '')
    return bwt


def FMIndex(bwt):
    fm = [{c: 0 for c in bwt}]
    for c in bwt:
        row = {symbol: count + 1 if (symbol == c) else count for symbol, count in fm[-1].iteritems()}
        fm.append(row)
    offset = {}
    N = 0
    for symbol in sorted(row.keys()):
        offset[symbol] = N
        N += row[symbol]
    return fm, offset

def getBWTTranscription(seq):
    ind = 0
    transcription = ""
    
    # iterate through the entire bwt
    while ind < len(seq):
        num = ""
        while seq[ind] not in codons:
            num += seq[ind]
            ind += 1
        if num != "":
            transcription += (seq[ind] * int(num))
        else:
            transcription += seq[ind]
        ind += 1
    return transcription


# THIS IS THE METHOD THAT IMPLEMENTS RETRIEVING A SUFFIX ARRAY FROM A BWT
def retrieveSuffixArray(bwt):
    # begin by generating the fm index and the offsets of the bwt inputted
    fm, offset = FMIndex(bwt)
    
    # generate a list to fill of the correct length
    arr = [0 for c in bwt]
    
    # we start by find the first item in the suffix array, the terminator
    predec = offset["$"] + fm[bwt.index("$")]["$"]
    for i in xrange(1, len(bwt)):
        arr[predec] = len(bwt) - i
        predec = offset[bwt[predec]] + fm[predec][bwt[predec]]
    
    return arr

# fetch the suffix array and the original text
# arr, text = retrieveSuffixArray(getBWTTranscription(loadBWT()))
arr = retrieveSuffixArray(getBWTTranscription(loadBWT()))

# finding the genomic indices of ATGACAACGC, GCGTTGTCAT, ATGACAACGCAT, ATGCGTTGTCAT
# this is the testing
print "ATGACAACGC is at", arr[lo1:hi1]
print "GCGTTGTCAT is at", arr[lo2:hi2]
print "ATGACAACGCAT is at", arr[lo3:hi3]
print "ATGCGTTGTCAT is at", arr[lo4:hi4]

# the results are
#
# ATGACAACGC is at 1006271 to 1006275 [2866253, 2253887, 299096, 4756596]
# GCGTTGTCAT is at [3840958, 4244113, 1945357, 3973758, 2706228, 1160902, 3509441, 3113230, 4720340, 
#                   1357533, 2939381, 3879426, 3884275]
# ATGACAACGCAT is at [299096]
# ATGCGTTGTCAT is at [3899847]


The suffix array is of length 4857433
ATGACAACGC is at 1006271 to 1006275 [2866253, 2253887, 299096, 4756596]
GCGTTGTCAT is at [3840958, 4244113, 1945357, 3973758, 2706228, 1160902, 3509441, 3113230, 4720340, 1357533, 2939381, 3879426, 3884275]
ATGACAACGCAT is at [299096]
ATGCGTTGTCAT is at [3899847]


### Click [here to submit](http://csbio.unc.edu/mcmillan/index.py?run=PS.upload) your completed problem set